In [ ]:
pip install backtrader

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 5.6 MB/s eta 0:00:00


In [ ]:
#Generating 1,0,-1 signals

import datetime
import numpy as np
import pandas as pd
import yfinance as yf

In [ ]:
data = pd.read_csv('/content/strategy.csv')
data = pd.read_csv('/content/strategy.csv')
data.reset_index(inplace=True)
data

,index,Unnamed: 0,datetime,open,high,low,close,volume,close\open,signal
0,0,0,2018-01-01,13715.65,13818.55,12750.00,13135.00,6970.441076,close,0
1,1,1,2018-01-02,13135.00,14750.00,12890.02,14580.00,16359.347340,close,0
2,2,2,2018-01-03,14576.87,15473.49,14150.00,14749.97,19457.238601,close,0
3,3,3,2018-01-04,14710.00,15280.00,13918.04,14811.00,18783.876561,close,0
4,4,4,2018-01-05,14805.01,17118.13,14600.00,16579.00,22383.657333,close,0
...,...,...,...,...,...,...,...,...,...,...
1468,1468,1468,2022-01-08,41707.04,42246.37,40501.00,40679.82,27031.900870,close,0
1469,1469,1469,2022-01-09,40679.82,42300.00,40520.00,41966.99,27291.009660,close,0
1470,1470,1470,2022-01-10,41966.99,42786.70,39650.00,41604.58,51064.904360,close,0
1471,1471,1471,2022-01-11,41604.58,43100.00,41158.99,42755.00,38062.818280,close,0


In [ ]:
data.datetime = pd.to_datetime(data.datetime,format="%d-%m-%Y %H:%M:%S",utc=True)

In [ ]:
import backtrader as bt
import datetime
import yfinance as yf
import datetime as dt


class PandasDataFeed(bt.feeds.PandasData):

        lines=('SIGNAL',)

        params = (

        ('SIGNAL', 'signal'),  # Add your signal column to the data feed, change the second 'signal ' corresponding to the name of signals column in your dataset

    )

data['datetime'] = pd.to_datetime(data['datetime'])
data.set_index('datetime', inplace=True)
data = PandasDataFeed(dataname=data,
                      fromdate=datetime.datetime(2018, 1, 1),
                      todate=datetime.datetime(2022, 1, 12)
                            )

In [ ]:
class CommInfoFractional(bt.CommissionInfo):
    def getsize(self, price, cash):
        '''Returns fractional size for cash operation @price'''
        return self.p.leverage * (cash / price)

In [ ]:
import collections

In [ ]:
#Final backtrader Framework. Takes in a dataframe and gives various parameters.
import backtrader.analyzers as btanalyzers

class FixedDollarSizer(bt.Sizer):
    params = (
        ('amount', 1000),  # Fixed dollar amount per trade
    )

    def _getsizing(self, comminfo, cash, data, isbuy):
        if isbuy:
            # Calculate the size based on the fixed dollar amount
            return self.p.amount/data.close[0]
        else:
            return self.broker.getposition(data).size

class MySignal(bt.Indicator):
    lines = ('signal',)
    i=1
    def log(self, txt, dt=None):
      dt = dt or self.datas[0].datetime.date(0)
      print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        self.lines.signal = self.data.SIGNAL
    def next(self):
        print(self.l.signal[0])


class MyStrategy(bt.Strategy):

    def log(self, txt, dt=None):
      dt = dt or self.datas[0].datetime.date(0)
      print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        self.percent_of_cash_per_trade = 1
        self.in_position = None  # Flag to track position
        self.trades=0
        self.netreturns=0
        self.entry_price=0
        self.profits=0
        self.profitable_trades=0
        self.loss_trades=0
        self.losses=0
        self.sizes=0
    '''
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None
'''

    def notify(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enougth cash
        if order.status in [order.Completed, order.Canceled, order.Margin]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
                self.opsize = order.executed.size
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

                gross_pnl = (order.executed.price - self.buyprice) * \
                    self.opsize


                net_pnl = gross_pnl - self.buycomm - order.executed.comm
                self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                         (gross_pnl, net_pnl))

    def next(self):
       #self.log(self.broker.cash)
        if self.in_position==None:

            if self.data.SIGNAL[0]==1:

                self.log(f'{self.broker.cash}')
                self.buy()
               # self.log(f'BUY LONG:{self.data.close[0]}')
                self.log(f'{self.broker.cash}')
                self.in_position='Long'
                self.entry_price=self.data.close[0]

            elif self.data.SIGNAL[0]==-1 :

                self.sizes = (1000 * self.percent_of_cash_per_trade) / self.data.close[0]
                self.sell(size=self.sizes)
                self.log(f'{self.broker.cash}')
                self.log(f'{self.trades}')
               # self.log(f'SELL SHORT:{self.data.close[0]}')
                self.in_position='Short'
                self.entry_price=self.data.close[0]
        elif self.in_position=='Long':
            if self.data.SIGNAL[0]==-1:

                self.close()
                self.log(f'{self.broker.cash}')
                self.trades+=1
                self.log(f'Trade Number:{self.trades},CLOSING LONG POSITION:{self.data.close[0]}')
                self.netreturns+=(self.data.close[0]-self.entry_price)*self.sizes
                if (self.data.close[0]-self.entry_price>0):
                  self.profits+=(self.data.close[0]-self.entry_price)*self.sizes
                  self.profitable_trades+=1
                elif (self.data.close[0]-self.entry_price<0):
                  self.losses+=(self.data.close[0]-self.entry_price)*self.sizes
                  self.loss_trades+=1
           #     self.log(f'Trade return={(self.data.close[0]-self.entry_price)}')
                self.in_position = None
            self.sizes=0
        elif self.in_position=='Short':
            if self.data.SIGNAL[0]==1:


                self.close()
                self.log(f'{self.broker.cash}')
                self.trades+=1
             #   self.log(f'Trade return={-(self.data.close[0]-self.entry_price)}')
                self.in_position = None
                self.sizes=0


def printTradeAnalysis(analyzer):
    #Get the results we are interested in
    total_open = analyzer.total.open
    total_closed = analyzer.total.closed
    total_won = analyzer.won.total
    total_lost = analyzer.lost.total
    win_streak = analyzer.streak.won.longest
    lose_streak = analyzer.streak.lost.longest
    pnl_net = round(analyzer.pnl.net.total,2)
    strike_rate = (total_won / total_closed) * 100
    #Designate the rows
    h1 = ['Total Open', 'Total Closed', 'Total Won', 'Total Lost']
    h2 = ['Strike Rate','Win Streak', 'Losing Streak', 'PnL Net']
    r1 = [total_open, total_closed, total_won, total_lost]
    r2 = [strike_rate, win_streak, lose_streak, pnl_net]
    #Check which set of headers is the longest.
    if len(h1) > len(h2):
        header_length = len(h1)
    else:
        header_length = len(h2)
    #Print the rows
    print_list = [h1,r1,h2,r2]
    row_format ="{:<15}" * (header_length + 1)
    print("Trade Analysis Results:")
    for row in print_list:
        print(row_format.format('',*row))


if __name__ == '__main__':
    cerebro = bt.Cerebro()

    cerebro.adddata(data)
    cerebro.broker.addcommissioninfo(CommInfoFractional())
    cerebro.addsizer(FixedDollarSizer)
    cerebro.broker.set_coc(True)
    #cerebro.add_signal(bt.SIGNAL_LONGSHORT, MySignal)
    cerebro.addstrategy(MyStrategy)
    cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='mysharpe', timeframe=bt.TimeFrame.Days,riskfreerate=0.03 )
    cerebro.addanalyzer(btanalyzers.DrawDown, _name='drawdown', fund=True)
    cerebro.addanalyzer(btanalyzers.TradeAnalyzer, _name='ta')
    cerebro.broker.set_cash(10000.0)
    cerebro.broker.setcommission(commission=0.001)
    #cerebro.addobserver(bt.observers.DrawDown)
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    thestrats=cerebro.run()
    thestrat = thestrats[0]
    printTradeAnalysis(thestrat.analyzers.ta.get_analysis())
    print('Sharpe Ratio:', thestrat.analyzers.mysharpe.get_analysis())
    print('Max Drawdown:', thestrat.analyzers.drawdown.get_analysis())
    #print('Net Returns:',thestrat.netreturns)
    #print('Profits:',thestrat.profits)
    #print('Losses:',thestrat.losses)
    #print('Profitable Trades:',thestrat.profitable_trades)
    #print('Loss Trades:',thestrat.loss_trades)
    #print('Total Closed Trades:',thestrat.trades)
    #print('Average Winning Trade:' ,thestrat.profits/thestrat.profitable_trades)
    #print('Average Losing Trade:' ,thestrat.losses/thestrat.loss_trades)
    print('')
    print('Ending Portfolio Value: %.2f' % cerebro.broker.getvalue())
    #print(MyStrategy.returns)


Starting Portfolio Value: 10000.00
2018-03-02, 10000.0
2018-03-02, 10000.0
2018-03-03, BUY EXECUTED, Price: 10909.99, Cost: 1000.00, Comm 1.00
2018-03-14, 8999.0
2018-03-14, Trade Number:1,CLOSING LONG POSITION:8250.17
2018-03-15, SELL EXECUTED, Price: 8250.17, Cost: 1000.00, Comm 0.76
2018-03-15, OPERATION PROFIT, GROSS -243.80, NET -245.55
2018-03-15, 9754.447056321775
2018-03-15, 1
2018-03-16, SELL EXECUTED, Price: 8126.00, Cost: -1000.00, Comm 1.00
2018-03-16, OPERATION PROFIT, GROSS -255.18, NET -257.18
2018-04-24, 10753.447056321775
2018-04-25, BUY EXECUTED, Price: 9410.99, Cost: -1000.00, Comm 1.16
2018-04-27, 9594.155770326204
2018-04-27, 9594.155770326204
2018-04-28, BUY EXECUTED, Price: 9230.89, Cost: 1000.00, Comm 1.00
2018-05-22, 8593.155770326204
2018-05-22, Trade Number:3,CLOSING LONG POSITION:8214.39
2018-05-23, SELL EXECUTED, Price: 8214.39, Cost: 1000.00, Comm 0.89
2018-05-23, OPERATION PROFIT, GROSS -110.12, NET -112.01
2018-05-23, 9482.146497114196
2018-05-23, 3
2018